In [ ]:
!pip install transformers torch




In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import csv

# Use on_bad_lines='skip' to skip problematic rows
data = pd.read_csv('train.csv', errors='replace')

# Display the first few rows
data.head()

TypeError: read_csv() got an unexpected keyword argument 'errors'

In [ ]:
print(data['chat'].head(10))


In [ ]:
import ast

def preprocess_data(data):
    prompts = []
    responses = []

    for entry in data['chat']:
        # Convert string representation of list to actual list
        try:
            entry_list = ast.literal_eval(entry)  # Safely evaluate the string representation
            for item in entry_list:
                prompts.append(item['prompt'])
                responses.append(item.get('response', ''))
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing entry: {entry} \nError: {e}")

    return prompts, responses

prompts, responses = preprocess_data(data)

print("Sample Prompts:", prompts[:5])
print("Sample Responses:", responses[:5])


In [ ]:
prompts, responses = preprocess_data(data)

print("Number of Prompts:", len(prompts))
print("Number of Responses:", len(responses))


In [ ]:
!pip install transformers


In [ ]:
import json

#Prepare the list to hold formatted entries
formatted_data = []

#Process each entry in the 'chat' column
for index, row in df.iterrows():
    chat = row['chat']
    if isinstance(chat, str) and ':' in chat:
        prompt, response = chat.split(':', 1)  # Splitting the prompt and response
        formatted_data.append({
            'prompt': prompt.strip(),     # Strip any leading/trailing whitespace
            'response': response.strip()   # Strip any leading/trailing whitespace
        })

with open('formatted_data.json', 'w') as json_file:
    json.dump(formatted_data, json_file, indent=4)

print("Data conversion completed. Check 'formatted_data.json' for the output.")


In [ ]:
formatted_data[:5]

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenized_data = [tokenizer.encode(entry['prompt'] + ' ' + entry['response'], return_tensors='pt') for entry in formatted_data]


In [ ]:
import torch
from torch.utils.data import Dataset

class StoryDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

story_dataset = StoryDataset(tokenized_data)


In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

def tokenize_and_pad(data):
    tokenized_data = []
    for entry in data:
        encoded = tokenizer(
            entry['prompt'] + ' ' + entry['response'],
            return_tensors='pt',
            padding='longest',
            truncation=True,
            max_length=512
        )
        tokenized_data.append(encoded['input_ids'].squeeze(0))

    padded_data = torch.nn.utils.rnn.pad_sequence(tokenized_data, batch_first=True, padding_value=tokenizer.pad_token_id)
    return padded_data

padded_data = tokenize_and_pad(formatted_data)

class StoryDataset(Dataset):
    def __init__(self, padded_data):
        self.data = padded_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

story_dataset = StoryDataset(padded_data)

train_loader = DataLoader(story_dataset, batch_size=2, shuffle=True)

num_epochs = 3
batch_limit = 10

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        if i >= batch_limit:
            break

        optimizer.zero_grad()

        attention_mask = (batch != tokenizer.pad_token_id).int()

        outputs = model(batch, attention_mask=attention_mask, labels=batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 1 == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")

    avg_loss = running_loss / batch_limit
    print(f"Epoch {epoch} finished. Average Loss: {avg_loss}")

print("Training completed.")


In [ ]:
model.save_pretrained("story_model")
tokenizer.save_pretrained("story_model")


In [ ]:

model.eval()  # Set the model to evaluation mode

def generate_story(prompt, max_length=400):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = (input_ids != tokenizer.pad_token_id).int()

    # Generate story using the model
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Prevent repetition of phrases
        early_stopping=True
    )

    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story

# Enhance the existing story based on user input
def enhance_story(existing_story, enhancement_prompt, max_length=150):
    input_text = existing_story + " " + enhancement_prompt
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=max_length + len(input_ids[0]),
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
    )
    enhanced_story = tokenizer.decode(output[0], skip_special_tokens=True)
    return enhanced_story

if __name__ == '__main__':
    existing_story = ""

    prompt = input("Enter a prompt for the story: ")
    existing_story = generate_story(prompt)  # Generate the first story
    print("\nGenerated Story:\n", existing_story.replace(".", ".\n\n"))

    while True:
        action_choice = input("\nDo you want to (1) enhance this story, (2) generate a new story, or (3) confirm this story? (Enter 1, 2, or 3): ").strip()

        if action_choice == '1':
            enhancement_prompt = input("Enter details to enhance the story: ")
            existing_story = enhance_story(existing_story, enhancement_prompt)
            print("\nEnhanced Story:\n", existing_story.replace(".", ".\n\n"))

        elif action_choice == '2':
            prompt = input("Enter a prompt for the new story: ")
            existing_story = generate_story(prompt)
            print("\nGenerated Story:\n", existing_story.replace(".", ".\n\n"))

        elif action_choice == '3':
            break

        else:
            print("Invalid input. Please enter 1, 2, or 3.")
            continue

        user_choice = input("\nDo you want to continue? (yes/no): ").strip().lower()
        if user_choice != 'yes':
            break

    print("Thank you for using the storytelling generator!")